In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchnlp.encoders.text import SpacyEncoder, pad_tensor
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from blitz.modules import BayesianEmbedding, BayesianLSTM, BayesianLinear

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.notebook import tqdm



In [ ]:
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

## Preprocessing

In [ ]:
#import
df = pd.read_csv('data/Reviews.csv')

#drop useless data
df = df.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Time', 'Summary',], axis=1)

#remove ambiguous 3 and 4 stars for balancing
#
df = df[df['Score'] != 3]

In [ ]:
#create labels and preprocess
df['Score'] = df['Score'].apply(lambda i: 'positive' if i > 4 else 'negative')
df['Text'] = df['Text'].apply(lambda x:x.lower())

#set names for beautiful df
df.columns = ['labels', 'text']

In [ ]:
idx_positive = df[df['labels']=='positive'].index
nbr_to_drop = len(df) - len(idx_positive)

drop_indices = np.random.choice(idx_positive, nbr_to_drop, replace=False)
df = df.drop(drop_indices)

In [ ]:
(df['labels'] =='negative').mean()

In [ ]:
text_as_list = df['text'].tolist()
labels_as_list = df['labels'].tolist()

In [ ]:
%%time
encoder = SpacyEncoder(text_as_list)

In [ ]:
%%time
encoded_texts = []
for i in tqdm(range(len(text_as_list))):
    encoded_texts.append(encoder.encode(text_as_list[i]))

In [ ]:
%%time
lengths = [len(i) for i in tqdm(encoded_texts)]

In [ ]:
length_as_series = pd.Series(lengths)
plt.title("Probability Density Function for text lengths")
sns.distplot(length_as_series)

In [ ]:
max_pad_length = length_as_series.quantile(0.9)

In [ ]:
max_pad_length = 70

In [ ]:
%%time
reviews = []
labels = []

for i in tqdm(range(len(encoded_texts))):
    if len(encoded_texts[i]) < max_pad_length:
        reviews.append(encoded_texts[i])
        labels.append(1 if labels_as_list[i] == "positive" else 0)
        
assert len(reviews) == len(labels), "The labels and feature lists should have the same time"

In [ ]:
%%time
padded_dataset = []
for i in tqdm(range(len(reviews))):
    padded_dataset.append(pad_tensor(reviews[i], int(max_pad_length)))

In [ ]:
#preparing the final dataset:
X = torch.stack(padded_dataset)
y = torch.tensor(labels)

In [ ]:
(y==1).float().mean()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.25,
                                                    random_state=42)

X_train, y_train = torch.tensor(X_train), torch.tensor(y_train)
X_test, y_test = torch.tensor(X_test), torch.tensor(y_test)

## Creating network and dataset

In [ ]:
import math

class CustomLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz):
        super().__init__()
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.W = nn.Parameter(torch.Tensor(input_sz, hidden_sz * 4))
        self.U = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bias = nn.Parameter(torch.Tensor(hidden_sz * 4))
        self.init_weights()
                
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
         
    def forward(self, x, 
                init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = (torch.zeros(bs, self.hidden_size).to(x.device), 
                        torch.zeros(bs, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states
         
        HS = self.hidden_size
        for t in range(seq_sz):
            x_t = x[:, t, :]
            # batch the computations into a single matrix multiplication
            gates = x_t @ self.W + h_t @ self.U + self.bias
            i_t, f_t, g_t, o_t = (
                torch.sigmoid(gates[:, :HS]), # input
                torch.sigmoid(gates[:, HS:HS*2]), # forget
                torch.tanh(gates[:, HS*2:HS*3]),
                torch.sigmoid(gates[:, HS*3:]), # output
            )
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(encoder.vocab)+1, 32)
        self.lstm = CustomLSTM(32,32)#nn.LSTM(32, 32, batch_first=True)
        self.fc1 = nn.Linear(32, 2)
        
    def forward(self, x):
        x_ = self.embedding(x)
        x_, (h_n, c_n) = self.lstm(x_)
        x_ = (x_[:, -1, :])
        x_ = self.fc1(x_)
        return x_

In [ ]:
ds_train = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(ds_train, batch_size=128, shuffle=True)

ds_test = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(ds_test, batch_size=256, shuffle=True)

In [ ]:
device = torch.device('cuda')
classifier = Net().to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.005)#0.002 dives 85% acc
criterion = nn.CrossEntropyLoss()

In [ ]:
epoch_bar = tqdm(range(10),
                 desc="Training",
                 position=0,
                 total=2)

acc = 0

for epoch in epoch_bar:
    batch_bar = tqdm(enumerate(train_loader),
                     desc="Epoch: {}".format(str(epoch)),
                     position=1,
                     total=len(train_loader))
    
    for i, (datapoints, labels) in batch_bar:
        
        optimizer.zero_grad()
        
        preds = classifier(datapoints.long().to(device))
        loss = criterion(preds, labels.to(device))
        loss.backward()
        optimizer.step()
        #acc = (preds.argmax(dim=1) == labels).float().mean().cpu().item()
        
        if (i + 1) % 50 == 0:
            acc = 0
            
            with torch.no_grad():
                for  i, (datapoints_, labels_) in enumerate(test_loader):
                    preds = classifier(datapoints_.to(device))
                    acc += (preds.argmax(dim=1) == labels_.to(device)).float().sum().cpu().item()
            acc /= len(X_test)

        batch_bar.set_postfix(loss=loss.cpu().item(),
                              accuracy="{:.2f}".format(acc),
                              epoch=epoch)
        batch_bar.update()

        
    epoch_bar.set_postfix(loss=loss.cpu().item(),
                          accuracy="{:.2f}".format(acc),
                          epoch=epoch)
    epoch_bar.update()